In [ ]:
import os, json
import numpy as np
import pandas as pd

from networkx import fruchterman_reingold_layout as layout

# GraphViz is arguably the best graph drawing tool, but it is old and tricky install.
# Uncommenting the line below line will get you slightly better layouts, if you can get it working...
# from networkx.drawing.nx_agraph import graphviz_layout as layout

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2
import hypernetx as hnx

# Data
Construct a hypergraph dictionary mapping edge id's to vertex sets

In [ ]:
scenes = {
    0: ('FN', 'TH'),
    1: ('TH', 'JV'),
    2: ('BM', 'FN', 'JA'),
    3: ('JV', 'JU', 'CH', 'BM'),
    4: ('JU', 'CH', 'BR', 'CN', 'CC', 'JV', 'BM'),
    5: ('TH', 'GP'),
    6: ('GP', 'MP'),
    7: ('MA', 'GP')
}

H = hnx.Hypergraph(scenes)

# Visualization
Use the default drawing tool to visualize `H` and its dual. This renders an Euler diagram of the hypergraph where vertices are black dots and hyper edges are convex shapes containing the vertices belonging to the edge set. It is not always possible to render a "correct" Euler diagram for an arbitrary hypergraph. This technique will lead to false positives, cases where a hyper edge incorrectly contains a vertex not belonging to its set.

In [ ]:
hnx.drawing.draw(H)

We can also view the dual of this graph easily, using the `H.dual()` command.

In [ ]:
hnx.drawing.draw(H.dual())

# Basic Visualization Parameters
We will now explore some of the basic visualization parameters that are special for hypergraph visualization
* collapsing vertices and edges
* label options

## Collapsing Vertices
By passing in a hypergraph with its nodes collapsed (using `H.collapse_nodes()`), we show nodes with identical hyper edge membership to be collapsed into a single dot. The drawing tool automatically detects if nodes and edges have been collapsed, and the dot is labeled with the list of nodes it represents. In this case, `{CN, CC, BR}` and `{CH, JU}` were collapsed. The size of the dot increases to reflect the number of members.

We will use a consistent random state across the next few diagrams to make the layout consistent.

In [ ]:
kwargs = {'layout_kwargs': {'seed': 39}}

hnx.drawing.draw(H.collapse_nodes(), **kwargs)

## Label options
For anything but trivially small hypergraphs, the number of items in a collapsed element can be large, causing the label to be very long. You can set `with_node_counts=True` to mitigate this. This will replae node labels with counts where there is more than one element at that node.

The collapsed nodes from above have been replaced with `x3` and `x2`, and the rest of the labels have disappeared.

In [ ]:
hnx.drawing.draw(H.collapse_nodes(), with_node_counts=True, **kwargs)

Similarly, hyper-edges can be collapsed and relabeled. We will use the dual to illustrate this.

In [ ]:
hnx.drawing.draw(H.dual().collapse_edges(), with_edge_counts=True)

## Disabling labels
If edge or node lables are a distraction, they can be turned off completely by passing in `with_edge_labels=False` or `with_node_labels=False`

In [ ]:
hnx.drawing.draw(H, with_edge_labels=False, **kwargs)

## Advanced Visualization Parameters
Properties like colors and thickness can be adjusted. This allows style parameters to be passed directly to the corresponding Matplotlib function. The mapping is as follows:
* `edges_kwargs` => `matplotlib.collections.PolyCollection`
* `nodes_kwargs` => `matplotlib.collections.PolyCollection`
* `edge_labels_kwargs` => `matplotlib.annotate`
* `node_labels_kwargs` => `matplotlib.annotate`

## Colors
By default, we cycle through one of 10 unique colors for edges. This can be overridden using the `edges_kwargs` parameter. Here we adjust the linewidth, edge color, and face color.

In [ ]:
hnx.drawing.draw(H.collapse_nodes(),
    edges_kwargs={
        'linewidths': 2,
        'edgecolors': 'brown',
        'facecolors': 'pink'
    },
    **kwargs
)

## Node colors
Pass an array of matplotlib colors to configure the individual colors of each node. The order of the array corresponds to the order returned by `H.__iter__()`.

In this example, we make the nodes that 

In [ ]:
H_collapsed = H.collapse_nodes()

hnx.drawing.draw(H_collapsed,
                 edges_kwargs={
                     'edgecolors': 'black'
                 },
                 nodes_kwargs={
                     'facecolors': ['red' if len(v) > 1 else 'black' for v in H_collapsed]
                 },
                 **kwargs)

## Edge colors
Pass an array of matplotlib colors to configure the individual colors of each edge. The order of the array corresponds to the order returned by `H.edges()`.

In [ ]:
cmap = plt.cm.viridis
alpha = .5

sizes = np.array([len(e) for e in H.edges()])
norm = plt.Normalize(sizes.min(), sizes.max())

hnx.drawing.draw(H,
                 label_alpha=0,
                 edges_kwargs={
                     'facecolors': cmap(norm(sizes))*(1, 1, 1, alpha),
                     'edgecolors': 'black',
                     'linewidths': 2
                 },
                 **kwargs)

## Font
Fontsize and other attributes can be set with the `node_labels_kwargs` and `edge_labels_kwargs` parameters. Here we make the font size large for illustrative purposes.

In [ ]:
hnx.drawing.draw(H.collapse_nodes(),
    node_labels_kwargs={
        'fontsize': 24
    },
    **kwargs
)

Font attributes can be individually set by passing in a dictionary mapping nodes to values

In [ ]:
hnx.drawing.draw(H,
    node_labels_kwargs={
        'fontsize': {v: 36 if v == 'JV' else 12 for v in H}
    },
    **kwargs
)